In [1]:
# Example of simple model load and evaluate

# ===============LICENSE_START=======================================================
# Apache-2.0
# ===================================================================================
# Copyright (C) 2019 AT&T Intellectual Property  All rights reserved.
# ===================================================================================
# This software file is distributed by AT&T
# under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# This file is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ===============LICENSE_END=========================================================


In [2]:
import numpy as np
import pandas as pd
import os,sys,shutil  # file checks
import dill as pickle   # serialize functions and data as compressed binary 
import gzip  # compression 
import yaml   # configuration file
import time  # time tracking

from acumos.modeling import Model, List, Dict, create_namedtuple, create_dataframe
from acumos.wrapped import load_model, _pack_pb_msg
from google.protobuf.json_format import MessageToJson, MessageToDict

import call_utils

# load our configutaion
config_path = 'config.yaml'
if not os.path.isfile(config_path):
    print("Sorry, can't find the configuration file {}, aborting.".format(config_path))
    sys.exit(-1)
config = yaml.safe_load(open(config_path))

In [3]:
## PART 1 - load and start a local model runner 
# https://pypi.org/project/acumos/#using-dataframes-with-scikit-learn

# read our larger datasets as binary files
with gzip.open(config["path"]["etl"], 'rb') as f:
    df = pickle.load(f)
print(df["X_test"].columns)
print(df["X_test"].sample(1).transpose())

Index(['helpful', 'reviewText', 'summary', 'unixReviewTime', 'categories',
       'description'],
      dtype='object')
                                                             4752
helpful                                                    [0, 0]
reviewText      As I'm typing this review, I'm looking over to...
summary                            Comfortable, Stylin' Mouse Pad
unixReviewTime                                         1306368000
categories      [office products, office & school supplies, de...
description     Add a vibrant, playful accent to your workspac...


In [4]:
# load model from disk, see that it is a nicely "wrapped" model
wrapped_model = load_model(os.path.join('data', config["publish"]["name_model2"]))
print(wrapped_model)

In [5]:


# actual evaluation code...

# test 1 - take an example of our own sample (where we can see format from the model)
X = [[[0,0], "I've had better", "not much", int(time.time()), ["office products"], "rubbery item"]]
call_utils.score_model(wrapped_model, X, True)



ReviewDataFrame(helpful=[0, 0], reviewText="I've had better", summary='not much', unixReviewTime=1553854818, categories=['office products'], description='rubbery item')
{'helpful': ['0', '0'], 'reviewText': "I've had better", 'summary': 'not much', 'unixReviewTime': '1553854818', 'categories': ['office products'], 'description': 'rubbery item'}
<bound method WrappedFunction.from_pb_bytes of <acumos.wrapped.WrappedFunction object at 0x115bff208>>
Preprocessing text column 'reviewText'...


TypeError: tuple indices must be integers or slices, not str

In [ ]:
# test 2 - evaluate our test data

# read our larger datasets as binary files
with gzip.open(config["path"]["etl"], 'rb') as f:
    df = pickle.load(f)
results = score_local_model(wrapped_model, df["X_test"].values, False)